<a href="https://colab.research.google.com/github/slavcho-k/vnp_zadaci/blob/master/Lab3(11_12_2023).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


###Во оваа лабораторика вежба фокусот е на time series предвидувања, односно предвидување на Close (BTC) вредноста во даденото податочно множество за Bitcoin. Ваша задача е со помош на decision tree и boosting методи да ја предвидите бараната вредност земајќи ги во предвид релевантните колони од податочното множество, но дополнително и вредноста на Close (BTC) во изминатите четири дена, бидејќи вредноста на Bitcoin-от во даден момент зависи од нејзината вредност во изминатите денови.

Hint : внимавајте како ќе го поделите множеството на train и test бидејќи се работи податоци во временска серија

##Симнете го податочното множество

In [2]:
! pip install gdown==v4.6.3

  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6


In [3]:
!gdown 14YlgjT_M8kDrvDKEv864fs7ZyI1lMErO

Downloading...
From: https://drive.google.com/uc?id=14YlgjT_M8kDrvDKEv864fs7ZyI1lMErO
To: /content/crypto_data.csv
100% 90.0k/90.0k [00:00<00:00, 4.02MB/s]


##Направете претпроцесирање на податоците

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('/content/crypto_data.csv')
df

,Date,Close (BNB),Close (BTC),Close (USDT),Close (ETH)
0,09-11-17,1.99077,7143.580078,1.00818,320.884003
1,10-11-17,1.79684,6618.140137,1.00601,299.252991
2,11-11-17,1.67047,6357.600098,1.00899,314.681000
3,12-11-17,1.51969,5950.069824,1.01247,307.907990
4,13-11-17,1.68662,6559.490234,1.00935,316.716003
...,...,...,...,...,...
1748,23-08-22,299.03000,21528.090000,1.00000,1662.770000
1749,24-08-22,296.45000,21395.020000,1.00000,1657.060000
1750,25-08-22,301.58000,21600.900000,1.00010,1696.460000
1751,26-08-22,279.60000,20260.020000,1.00000,1507.780000


In [6]:
df.isnull().sum()

Date            0
Close (BNB)     0
Close (BTC)     0
Close (USDT)    0
Close (ETH)     0
dtype: int64

In [7]:
df = df.set_index('Date')
df = df.sort_index()

In [16]:
lag = 4
for i in range(1, lag + 1):
  df[f'Close_prev_BTC{i}'] = df['Close (BTC)'].shift(i)

<ipython-input-16-dde41dba8d47>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'Close_prev_BTC{i}'] = df['Close (BTC)'].shift(i)
<ipython-input-16-dde41dba8d47>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'Close_prev_BTC{i}'] = df['Close (BTC)'].shift(i)
<ipython-input-16-dde41dba8d47>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [17]:
df

,Close (BNB),Close (BTC),Close (USDT),Close (ETH),Close_prev_BTC1,Close_prev_BTC2,Close_prev_BTC3,Close_prev_BTC4
Date,,,,,,,,
01-01-18,8.414610,13657.200200,1.007280,772.640991,NaN,NaN,NaN,NaN
01-01-19,6.075273,3843.520020,1.018335,140.819412,13657.200200,NaN,NaN,NaN
01-01-20,13.689083,7200.174316,0.999836,130.802002,3843.520020,13657.200200,NaN,NaN
01-01-21,37.905010,29374.152340,1.001916,730.367554,7200.174316,3843.520020,13657.200200,NaN
01-01-22,527.352722,47686.812500,1.000500,3769.697021,29374.152340,7200.174316,3843.520020,13657.200200
...,...,...,...,...,...,...,...,...
31-12-17,8.635580,14156.400390,1.012260,756.732971,61318.957030,13780.995120,9199.584961,6317.609863
31-12-18,6.164732,3742.700439,1.013722,133.368256,14156.400390,61318.957030,13780.995120,9199.584961
31-12-19,13.727642,7193.599121,0.999679,129.610855,3742.700439,14156.400390,61318.957030,13780.995120


In [18]:
df.isnull().sum()

Close (BNB)        0
Close (BTC)        0
Close (USDT)       0
Close (ETH)        0
Close_prev_BTC1    1
Close_prev_BTC2    2
Close_prev_BTC3    3
Close_prev_BTC4    4
dtype: int64

In [19]:
df = df.dropna(axis=0)
df

,Close (BNB),Close (BTC),Close (USDT),Close (ETH),Close_prev_BTC1,Close_prev_BTC2,Close_prev_BTC3,Close_prev_BTC4
Date,,,,,,,,
01-01-22,527.352722,47686.812500,1.000500,3769.697021,29374.152340,7200.174316,3843.520020,13657.200200
01-02-18,9.515840,9170.540039,0.986939,1036.790039,47686.812500,29374.152340,7200.174316,3843.520020
01-02-19,6.562617,3487.945313,1.006309,107.609787,9170.540039,47686.812500,29374.152340,7200.174316
01-02-20,18.218321,9392.875000,1.001216,183.673950,3487.945313,9170.540039,47686.812500,29374.152340
01-02-21,51.607697,33537.175780,1.000645,1369.040527,9392.875000,3487.945313,9170.540039,47686.812500
...,...,...,...,...,...,...,...,...
31-12-17,8.635580,14156.400390,1.012260,756.732971,61318.957030,13780.995120,9199.584961,6317.609863
31-12-18,6.164732,3742.700439,1.013722,133.368256,14156.400390,61318.957030,13780.995120,9199.584961
31-12-19,13.727642,7193.599121,0.999679,129.610855,3742.700439,14156.400390,61318.957030,13780.995120


In [20]:
features = [f'Close_prev_BTC{i}' for i in range(1, lag + 1)]

In [23]:
X, Y = df[features], df['Close (BTC)']
X

,Close_prev_BTC1,Close_prev_BTC2,Close_prev_BTC3,Close_prev_BTC4
Date,,,,
01-01-22,29374.152340,7200.174316,3843.520020,13657.200200
01-02-18,47686.812500,29374.152340,7200.174316,3843.520020
01-02-19,9170.540039,47686.812500,29374.152340,7200.174316
01-02-20,3487.945313,9170.540039,47686.812500,29374.152340
01-02-21,9392.875000,3487.945313,9170.540039,47686.812500
...,...,...,...,...
31-12-17,61318.957030,13780.995120,9199.584961,6317.609863
31-12-18,14156.400390,61318.957030,13780.995120,9199.584961
31-12-19,3742.700439,14156.400390,61318.957030,13780.995120


##Поделето го на тренирачко и тестирачко множество во сооднос 80:20

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

##Искористете соодветен Boosting модел за проблемот и тестирајте го моделот на тестирачкото множество

In [32]:
from xgboost import XGBRegressor

model = XGBRegressor(objective ='reg:linear',
    colsample_bytree = 0.3, learning_rate = 0.1,
    max_depth = 5, alpha = 2, n_estimators = 10)

model.fit(X_train, Y_train)
y_pred_xgb = model.predict(X_test)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:38:50] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [33]:
mean_squared_error(Y_test, y_pred_xgb)

157122851.94547525

##Искористете соодветен Decision Tree модел за проблемот и тестирајте го моделот на тестирачкото множество

In [26]:
from sklearn.tree import DecisionTreeRegressor

regressor = DecisionTreeRegressor(max_depth=100)

regressor.fit(X_train, Y_train)
y_pred = regressor.predict(X_test)

In [28]:
from sklearn.metrics import mean_squared_error

mean_squared_error(Y_test, y_pred)

30980979.865430962

In [29]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=1500, criterion='squared_error', max_depth=10)

rf.fit(X_train, Y_train)
y2_pred = rf.predict(X_test)

In [30]:
mean_squared_error(Y_test, y2_pred)

17301497.048674155

##Споредете ги резултатите добиени од двата модели со соодветни метрики

In [40]:
from sklearn.metrics import r2_score

r2_decision_tree = r2_score(Y_test, y_pred)
r2_random_forest = r2_score(Y_test, y2_pred)
r2_xgb = r2_score(Y_test, y_pred_xgb)

print("R2 Score - Decision Tree: {:.4f}".format(r2_decision_tree))
print("R2 Score - Random Forest: {:.4f}".format(r2_random_forest))
print("R2 Score - XGBoost: {:.4f}".format(r2_xgb))

R2 Score - Decision Tree: 0.8908
R2 Score - Random Forest: 0.9390
R2 Score - XGBoost: 0.4464


##Bonus Task: Направете визуелизации кои го илустрираат процесот на предвидување на двата горенавередени модели

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

fig, ax = plt.subplots()

ax.plot(y_pred, label='y_pred', marker='o')
ax.plot(y2_pred, label='y2_pred', marker='o')
ax.plot(y_pred_xgb, label='y_pred_xgb', marker='o')

ax.set_xlabel('Sample Index')
ax.set_ylabel('Values')
ax.set_title('Comparison of Predictions from Different Models')

ax.legend()

plt.show()